In [9]:
import torch
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU available")

NVIDIA A800 80GB PCIe


In [2]:
pip install modelscope

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/17/d8/f808e32ed7fa86617b9ac7a37b7dcff894c839108c4871cc33ffc4e65b7d/datasets-2.13.0-py3-none-any.whl (485 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.18.0
    Uninstalling datasets-2.18.0:
      Successfully uninstalled datasets-2.18.0

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from modelscope import snapshot_download, AutoModel, AutoTokenizer
model_dir  = snapshot_download('qwen/Qwen2.5-7B-Instruct', cache_dir='/root/tf-logs', revision='master')

Downloading:  43%|████▎     | 1.56G/3.60G [00:57<00:55, 39.7MB/s]2025-08-24 23:39:40,241 - modelscope - WARNING - Download file from: 1342177280 to: 1509949439 failed, will retry
2025-08-24 23:39:43,822 - modelscope - WARNING - Download file from: 1845493760 to: 2013265919 failed, will retry
2025-08-24 23:39:52,164 - modelscope - WARNING - Download file from: 2013265920 to: 2181038079 failed, will retry
Downloading:  48%|████▊     | 1.72G/3.60G [01:13<02:19, 14.5MB/s]2025-08-24 23:39:55,551 - modelscope - WARNING - Download file from: 2181038080 to: 2348810239 failed, will retry
Downloading:  74%|███████▍  | 2.66G/3.60G [01:43<00:38, 26.5MB/s]2025-08-24 23:40:26,017 - modelscope - WARNING - Download file from: 2181038080 to: 2348810239 failed, will retry
Downloading:  91%|█████████▏| 3.29G/3.60G [02:07<00:13, 25.5MB/s]2025-08-24 23:40:56,330 - modelscope - WARNING - Download file from: 3523215360 to: 3690987519 failed, will retry
2025-08-24 23:40:58,403 - modelscope - WARNING - Downloa

In [ ]:
from fastapi import FastAPI, Request
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import uvicorn
import json
import datetime
import torch

# 设置设备参数
DEVICE = "cuda"  # 使用CUDA
DEVICE_ID = "0"  # CUDA设备ID，如果未设置则为空
CUDA_DEVICE = f"{DEVICE}:{DEVICE_ID}" if DEVICE_ID else DEVICE  # 组合CUDA设备信息

# 清理GPU内存函数
def torch_gc():
    if torch.cuda.is_available():  # CUDA
        with torch.cuda.device(CUDA_DEVICE):  CUDA设备
            torch.cuda.empty_cache()  # 清空缓存
            torch.cuda.ipc_collect()  # 收集内存碎片

# 创建FastAPI应用
app = FastAPI()

# 处理POST请求的端点
@app.post("/")
async def create_item(request: Request):
    global model, tokenizer  # 在函数内部使用模型和分词器
    json_post_raw = await request.json()  # 获取POST请求的JSON数据
    json_post = json.dumps(json_post_raw)  # 将JSON数据转换为字符串
    json_post_list = json.loads(json_post)  # 将字符串转换为Python对象
    prompt = json_post_list.get('prompt')  # 获取请求中的提示

    messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
    ]

    # 调用模型进行对话生成
    input_ids = tokenizer.apply_chat_template(messages,tokenize=False,add_generation_prompt=True)
    model_inputs = tokenizer([input_ids], return_tensors="pt").to('cuda')
    generated_ids = model.generate(model_inputs.input_ids,max_new_tokens=512)
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    now = datetime.datetime.now()  # 获取当前时间
    time = now.strftime("%Y-%m-%d %H:%M:%S")  # 格式化时间为字符串
    # 构建响应JSON
    answer = {
        "response": response,
        "status": 200,
        "time": time
    }
    # 构建日志信息
    log = "[" + time + "] " + '", prompt:"' + prompt + '", response:"' + repr(response) + '"'
    print(log)  # 
    torch_gc()  # 
    return answer  # 

# 主函数入口
if __name__ == '__main__':
    # 加载预训练的分词器和模型
    model_name_or_path = '/root/tf-logs/qwen/Qwen2.5-7B-Instruct'
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=False)
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto", torch_dtype=torch.bfloat16)

    
    uvicorn.run(app, host='0.0.0.0', port=6006, workers=1)  

In [5]:
pip install transformers==4.44.2

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install huggingface-hub==0.25.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install accelerate==0.34.2

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install modelscope==1.18.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install langchain==0.3.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import requests
import json

def get_completion(prompt):
    headers = {'Content-Type': 'application/json'}
    data = {"prompt": prompt}
    response = requests.post(url='http://127.0.0.1:6006', headers=headers, data=json.dumps(data))
    return response.json()['response']

if __name__ == '__main__':
    print(get_completion('你好'))

你好！有什么可以帮助你的吗？


In [10]:
import requests
import json

def get_completion(prompt):
    headers = {'Content-Type': 'application/json'}
    data = {"prompt": prompt}
    response = requests.post(url='http://127.0.0.1:6006', headers=headers, data=json.dumps(data))
    return response.json()['response']

if __name__ == '__main__':
    # 向模型发送"我想加入HKUST AI for Chemistry Lab"
    print(get_completion('我想加入HKUST AI for Chemistry Lab'))

加入香港科技大学（HKUST）的AI for Chemistry实验室是一个很好的决定，该实验室专注于利用人工智能技术解决化学领域的挑战。要加入这个实验室，你可以按照以下步骤进行：

1. **了解实验室信息**：首先，你需要详细了解HKUST的AI for Chemistry Lab的研究方向、团队成员和正在进行的项目。这有助于你确定自己是否对该领域感兴趣，并且能够找到与你的研究兴趣相匹配的机会。

2. **准备个人资料**：准备一份详细的个人简历或CV，包括你的教育背景、科研经历、发表的文章等。如果你有相关的科研项目经验或者参加过相关的竞赛，也应该在简历中突出显示。

3. **联系导师**：大多数情况下，加入实验室需要通过申请过程。你可以访问HKUST官网查找相关实验室的信息，然后通过邮件或其他方式直接联系实验室的负责人或导师，表达你的兴趣并询问如何申请。

4. **申请材料**：除了个人简历外，你可能还需要提供推荐信、个人陈述或研究计划等材料。这些材料可以帮助实验室了解你的研究兴趣、目标以及为什么你适合他们的团队。

5. **面试准备**：如果被邀请参加面试，那么提前做好准备是非常重要的。了解实验室的研究方向，准备好回答关于你的研究经历、兴趣以及为什么想加入这个实验室的问题。

6. **遵守申请截止日期**：确保在提交所有申请材料之前仔细检查，不要错过任何重要的截止日期。

加入这样的实验室不仅可以获得宝贵的研究经验，还有机会与其他优秀的学生和研究人员合作，共同推动科学研究的发展。祝你好运！


In [1]:
import requests
import json

def get_completion(prompt):
    headers = {'Content-Type': 'application/json'}
    data = {"prompt": prompt}
    response = requests.post(url='http://127.0.0.1:6006', headers=headers, data=json.dumps(data))
    return response.json()['response']

if __name__ == '__main__':
    # 向模型发送"我想加入HKUST AI for Chemistry Lab"
    print(get_completion('我正在积极寻求参与科研的机会!'))

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=6006): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f5e5831b3d0>: Failed to establish a new connection: [Errno 111] Connection refused'))